In [2]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 전역 설정
from matplotlib import rc
import os
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams["font.size"] = 15

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

In [3]:
# 경로 설정
data_path = '../../data'
saved_path = './saved'
output_path = './submission'

In [4]:
# 데이터 불러오기 
history_df = pd.read_csv(os.path.join(data_path, 'history_data.csv'), encoding='utf-8')
watch_e_df = pd.read_csv(os.path.join(data_path, 'watch_e_data.csv'), encoding='utf-8')

meta_df = pd.read_csv(os.path.join(data_path, 'meta_data.csv'), encoding='utf-8')
meta_plus_df = pd.read_csv(os.path.join(data_path, 'meta_data_plus.csv'), encoding='utf-8')

profile_df = pd.read_csv(os.path.join(data_path, 'profile_data.csv'), encoding='utf-8')

In [5]:
history_df = history_df.drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).reset_index(drop = True)
watch_e_df = watch_e_df.drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).reset_index(drop = True)

# 5.meta data (meta_data &meta_plus_data)

In [13]:
print('meta data 개수:',len(meta_df),'meta plus data 개수:',len(meta_plus_df))
print('meta 와 meta_plus album_id 다른지 확인:', set(meta_df.album_id)-set(meta_plus_df.album_id), set(meta_plus_df.album_id)-set(meta_df.album_id))

meta data 개수: 42602 meta plus data 개수: 767948
meta 와 meta_plus album_id 다른지 확인: set() set()


- 서로 album_id 집합이 정확하게 일치 확인
- album_id 기준으로 merge 작업 가능 확인

In [8]:
len(set(meta_df.album_id)),len(set(meta_plus_df.album_id))

(39875, 39875)

- 메타 데이터 sub_title 변경되면서 중복 앨범이 생성됨
- 메타 플러스 데이터 태그가 여러개 생성되면서 중복 앨범이 생성됨

### 5-1 시청 log 데이터와 meta 데이터 album 기준으로 비교

In [9]:
print('전체 컨테츠 중 시청 기록 데이터에 없는 컨텐츠 수:',len(set(meta_df.album_id)-set(history_df.album_id)))
print('전체 컨텐츠 중 시청 기록에만 있는 컨텐츠 수:', set(history_df.album_id)- set(meta_df.album_id))

전체 컨테츠 중 시청 기록 데이터에 없는 컨텐츠 수: 19180
전체 컨텐츠 중 시청 기록에만 있는 컨텐츠 수: set()


- 하예진님 말씀하셨던 전체 컨테츠 중 로그 기록에 없는 컨텐츠를 학인함
- 시작 데이터에는 19180 컨텐츠가 콜드임
- 메타 데이터에 없는 컨텐츠가 시작 데이터에 있지는 않음

In [10]:
print('전체 컨테츠 중 종료 기록 데이터에 없는 컨텐츠 수:', len(set(meta_df.album_id)-set(watch_e_df.album_id)))
print('전체 컨텐츠 중 종료 기록에만 있는 컨텐츠 수:',set(watch_e_df.album_id) - set(meta_df.album_id))

전체 컨테츠 중 종료 기록 데이터에 없는 컨텐츠 수: 19514
전체 컨텐츠 중 종료 기록에만 있는 컨텐츠 수: set()


- 종료 데이터에는 19514 컨텐츠가 콜드
- 메타 데이터에 없는 컨텐츠가 종료 데이터에 있지는 않음

# 5-2 메타 데이터 merge 작업
- mete_df 경우 sub_title 이 같은 album_id 여도 다른 경우가 존재 (이것 때문에 데이터 개수가 늘어난 것!!)
- 따라서 같은 album_id의 sub_title은 리스트에 담아서 정상적인 데이터 개수로 만들어 줍니다.

## 1) 메타 데이터 sub_title 리스트 담기
- album_id 별 컬럼 값들이 다른 것이 있는 지 컬럼별 확인
- sub_title, onair_date, cast_7 컬럼이 같은 album_id 다른 값이 존재하는 컬럼
- 회의 결과: onari_date를 가지고 확인할 수 있는 것이 많이 없음 (중간 off 되었다는 것을 어떻게 찾을것인가!)   
cast_7는 개수가 몇개 없으며 다른 cast에 있을 가능성을 생각하여 전처리 예정
- sub_title 대해서만 리스트에 담는 전처리 진행하자
- 나중에 onehot로 표현하기 위해!
- sub_title를 리스트에 담기 위한 딕셔너리를 만들어줍니다.
- ex) {album_id:[sub_title1, sub_title2]}
- 이후 album_id 담아줍니다.

In [78]:
# 확인을 위한 함수입니다.(전처리 x)
# album_id 별 각 컬럼의 값이 여러개 인 경우의 컬럼 확인
def meta_df_preprocess(meta_df, column):
    meta_df_genre_large_list_dict= {}

    for idx, row in meta_df.iterrows():
        if meta_df_genre_large_list_dict.get(row['album_id'], None) == None:
            meta_df_genre_large_list_dict[row['album_id']] = [row[column]]
        elif row[column] not in meta_df_genre_large_list_dict[row['album_id']]:
            meta_df_genre_large_list_dict[row['album_id']] = meta_df_genre_large_list_dict[row['album_id']]+[row[column]]
    
    cnt =0
    # 같은 album_id에 다른 값이 있는 컬럼 표시
    
    for idx, i in enumerate(meta_df_genre_large_list_dict.values()):
        if (len(i) > 1) & (cnt==0):
            # 컬럼, 다른 값이 있는 개수를 출력
            print(column, ':',len(i))
            cnt +=1
        
    return meta_df_genre_large_list_dict

for column in ['title','sub_title','genre_large','genre_mid','genre_small','country','run_time','onair_date','cast_1','cast_2','cast_3','cast_4',\
    'cast_5','cast_6','cast_7']:
    meta_df_preprocess(meta_df,column)

sub_title : 2
onair_date : 2
cast_7 : 2


In [105]:
# sub_title 리스트 담기
meta_df_sub_title_list_dict= {}

for idx, row in meta_df.iterrows():
    if meta_df_sub_title_list_dict.get(row['album_id'], None) == None:
        meta_df_sub_title_list_dict[row['album_id']] = [row['sub_title']]
    elif row['sub_title'] not in meta_df_sub_title_list_dict[row['album_id']]:
        meta_df_sub_title_list_dict[row['album_id']] = meta_df_sub_title_list_dict[row['album_id']]+[row['sub_title']]
        
print('전체 컨텐츠 개수:',len(set(meta_df.album_id)),'딕셔너리 담긴 아이템 개수:',len(meta_df_sub_title_list_dict))        

meta_df_prepro_sub_title = meta_df.copy()
meta_df_prepro_sub_title['sub_title'] = meta_df_prepro_sub_title['album_id'].apply(lambda x: meta_df_sub_title_list_dict.get(x, None))

전체 컨텐츠 개수: 39875 딕셔너리 담긴 아이템 개수: 39875


- sub_title 리스트에 담기

In [106]:
# 전체 중복 삭제하기
meta_df_prepro_sub_title = meta_df_prepro_sub_title.drop_duplicates(subset=['album_id'], ignore_index=True)

print('전체 데이터수와 album_id 개수 동일 확인:', len(meta_df_prepro_sub_title), len(meta_df_prepro_sub_title.album_id.unique()))

전체 데이터수와 album_id 개수 동일 확인: 39875 39875


- 중복 삭제
- album_id 기준으로 
- 39875 동일 한 것 확인

### 2) album_id 와 title 서로 동일한가 (전처리 x, 확인 자료)
- 만약, 동일한 데이터가 있다면 같은 컨텐츠인데 여러개 생성된 것므로 확인!
- 이상치는 제거해야함으로
- 결론적으로는 다른 album_id 의 title 같은 것이 존재
- 이는 프로그램 회차 차인 인 것으로 판단 중
- 따라서 album_id 만이 고유 식별자인 것으로 봐야 함

## 3) 메타 플러스 데이터 keyword(keyword_name, type)리스트 담기

- meta plus 데이터는 동일 album_id 기준 keyword_type, keyword_name 컬러 값이 다른 것이 존재
- keyword_type, keyword_name 개수가 서로 다른 것 확인 dict로 서로 개수 맞춰주자
- 이후 keyword_type, keyword_name 모두 리스트 담아서 전처리 진행

In [69]:
len(set(meta_plus_df.keyword_type)), len(set(meta_plus_df.keyword_name))
# 39개 정도 차임

(1101, 1062)

- type 과 name 수가 다름 이는 같은 name 여도 type이 다른 것이 있는 것으로 보임
- name 만 사용해도 되지만 나중 한글은 입력 및 서칭하는데 문제가 있을 수 있기 때문에 type 도 같은 수로 만들어 주자

In [92]:
meta_plus_type_name = meta_plus_df.copy()

In [93]:
type_to_name = {}

for idx, row in meta_plus_df.iterrows():
    type_to_name[row['keyword_type']] = row['keyword_name']
    
meta_plus_type_name['keyword_name'] = meta_plus_type_name['keyword_type'].apply(lambda x: type_to_name[x])    

name_to_type = {}
for idx, row in meta_plus_df.iterrows():
    name_to_type[row['keyword_name']] = row['keyword_type']
    
meta_plus_type_name['keyword_type'] = meta_plus_type_name['keyword_name'].apply(lambda x: name_to_type[x])
print('keyword_name, keyword_type 개수 확인:',len(set(meta_plus_type_name.keyword_name)), len(set(meta_plus_type_name.keyword_type)))
print('앨범 당 키워드 중복 여부 확인:',len(meta_plus_df),len(meta_plus_df.drop_duplicates(subset=['album_id','keyword_type'])))

keyword_name, keyword_type 개수 확인: 1062 1062
앨범 당 키워드 중복 여부 확인: 767948 767948


- 중복 없음
- 메타 데이터 sub_title 리스트에 담기

In [94]:
meta_plus_df_prepro_keyword = meta_plus_type_name.copy()

### 4) meta plus data type name 동일하게 만들기
- type, name 1062 로 동일하게 만들었으면 이제 album_id   
마다 중복된 type,name을 리스트에 채워 넣어서 unique한 album만 나오게 하자

In [96]:
meta_df_keyword_name_list_dict= {}
meta_df_keyword_type_list_dict= {}

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_name_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_name_list_dict[row['album_id']] = [row['keyword_name']]
    elif row['keyword_name'] not in meta_df_keyword_name_list_dict[row['album_id']]:
        meta_df_keyword_name_list_dict[row['album_id']] = meta_df_keyword_name_list_dict[row['album_id']]+[row['keyword_name']]

for idx, row in meta_plus_df_prepro_keyword.iterrows():
    if meta_df_keyword_type_list_dict.get(row['album_id'], None) == None:
        meta_df_keyword_type_list_dict[row['album_id']] = [row['keyword_type']]
    elif row['keyword_type'] not in meta_df_keyword_type_list_dict[row['album_id']]:
        meta_df_keyword_type_list_dict[row['album_id']] = meta_df_keyword_type_list_dict[row['album_id']]+[row['keyword_type']]        

meta_plus_df_prepro_keyword['keyword_name'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_name_list_dict.get(x, None))
meta_plus_df_prepro_keyword['keyword_type'] = meta_plus_df_prepro_keyword['album_id'].apply(lambda x: meta_df_keyword_type_list_dict.get(x, None))

meta_plus_df_keyword = meta_plus_df_prepro_keyword.drop_duplicates(subset=['album_id'],ignore_index=True).drop(columns=['keyword_value'])

### 5) meta data merge
- meta_df_prepro_sub_title : sub_title 전처리한 메타 데이터
- meta_plus_df_keyword : keyword name과 type 전처리한 메타 플러스 데이터
- 두개 meta 데이터 merge 진행

In [103]:
print('서로 중복 데이터 수:',len(set(meta_df_prepro_sub_title.album_id) - set(meta_plus_df_keyword.album_id)))
print('데이터 수 확인:',len(meta_df_prepro_sub_title), len(meta_plus_df_keyword.album_id))

서로 중복 데이터 수: 0
데이터 수: 39875 39875


In [78]:
all_meta_df = pd.merge(meta_df_sub_title, meta_plus_df_keyword, how='left',on='album_id')

In [79]:
all_meta_df.tail()

,album_id,title,sub_title,genre_large,genre_mid,genre_small,country,run_time,onair_date,cast_1,cast_2,cast_3,cast_4,cast_5,cast_6,cast_7,keyword_type,keyword_name
39870,39872,로티프렌즈와 색칠놀이! - 캔디코 ＆ 동화책,[로티프렌즈 미술놀이],키즈,놀이교실,NaN,한국,443,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010202, 05020203, 05030701, 05030...","[시리즈, 2분 이상 10분 미만, 엔터, 미술, 그림그리기/색칠하기, 언어1, 운동]"
39871,39873,로티프렌즈와 색칠놀이! - 그리피 ＆ 사탕 바구니,[로티프렌즈 미술놀이],키즈,놀이교실,NaN,한국,477,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010202, 05020203, 05030701, 05030...","[시리즈, 2분 이상 10분 미만, 엔터, 미술, 그림그리기/색칠하기, 언어1, 운동]"
39872,39874,로티프렌즈와 색칠놀이! - 베블리 ＆ 꽃,[로티프렌즈 미술놀이],키즈,놀이교실,NaN,한국,466,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010202, 05020203, 05030701, 05030...","[시리즈, 2분 이상 10분 미만, 엔터, 미술, 그림그리기/색칠하기, 언어1, 운동]"
39873,4779,손가락을 빨게 돼요,"[4-5세, 아이맘콕]",키즈,책,NaN,한국,293,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[05010102, 05010104, 05010202, 05010402, 05010...","[단편, 책TV, 2분 이상 10분 미만, 4세, 5세, 6세, 7세, 엔터, 동화..."
39874,11629,손가락 하나,[베이비 타요 동요2],키즈,노래율동,NaN,한국,91,NaN,타요와 친구들,NaN,NaN,NaN,NaN,NaN,NaN,"[05010101, 05010201, 05020203, 05020205, 05020...","[시리즈, 2분 미만, 엔터, 노래, 율동, 음악, 동요, 노래 따라하기, 신나는,..."
